# Basics

In [2]:
#eda stacks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import s3fs
import os
import sys

#display settings
pd.set_option("display.max_columns", 200)      # show all columns
pd.set_option("display.max_rows", 20)         # show more rows
pd.set_option("display.width", 1200)           # wider output
pd.set_option("display.max_colwidth", 100)     # long text columns
pd.set_option("display.float_format", "{:.2f}".format)  # clean decimals
sns.set_theme(style="whitegrid")

In [3]:
bucket="kerala-ayurveda-s3"
prefix="bronze-tier/bronze_data.csv"
s3_path=f"s3://{bucket}/{prefix}"
df=pd.read_csv(s3_path)

# Analysis

1. Monthly KPI snapshot
  * sessions, users
  * product view rate, add-to-cart rate, checkout-start rate
  * purchase CVR (session → purchase)
  * revenue + AOV 

In [4]:
df['month']=pd.to_datetime(df['date_ist']).dt.to_period('M')

##### Flattening the JSONs into respective Columns

In [5]:
#lists have to be exploded and then normalized, dicts can be just normlaized
import json
import ast

def parse_json_safe(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except:
            try:
                return ast.literal_eval(x)
            except:
                return None
    return None


##### Sessions & Users

In [6]:
df['purchase_parsed'] = df['purchase'].apply(parse_json_safe)
df['dimensions_parsed'] = df['dimensions'].apply(parse_json_safe)

dim_df = (
    pd.json_normalize(df['dimensions_parsed'])
    .reindex(df.index)
)
purchase_df = (
    pd.json_normalize(df['purchase_parsed'])
    .reindex(df.index)
)
df_flat = pd.concat(
    [
        df.drop(columns=[
            'purchase',
            'purchase_parsed',
            'dimensions',
            'dimensions_parsed'
        ]),
        dim_df,
        purchase_df
    ],
    axis=1
)

In [7]:
df_flat.sample()

,date_ist,user_id,session_id,funnels,month,landing_page,landing_page_type,source,medium,campaign,device,orders,orders_dups_inclusive,rev,aov
326301,2025-12-23,1473995165.18,1766468386,"{'product_view': {'flag': 1, 'first_timestamp': '2025-12-23 11:09:46 UTC+0530'}, 'add_shipping_i...",2025-12,https://keralaayurveda.com/,homepage,google,cpc,Google_PMax_Selected_Products_12072025,mobile,0.00,0.00,0.00,0.00


In [8]:
df_flat.rev.sum()

np.float64(5483932.51)

In [9]:
funnel=df_flat[['month','date_ist','user_id','session_id','funnels']]
rest=df_flat.drop(columns=['funnels'])

In [10]:
funnel['funnels_parsed'] = funnel['funnels'].apply(parse_json_safe)
funnel_norm = pd.json_normalize(
    funnel['funnels_parsed'],
    sep='_'
)

/tmp/ipykernel_7617/4223611394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funnel['funnels_parsed'] = funnel['funnels'].apply(parse_json_safe)


In [11]:
funnel_norm.columns = [str(c) for c in funnel_norm.columns]
funnel_norm = funnel_norm.reindex(funnel.index)
funnel_flat = pd.concat(
    [
        funnel.drop(columns=['funnels_parsed']),
        funnel_norm
    ],
    axis=1
)


In [12]:
funnel_flat.sample()

,month,date_ist,user_id,session_id,funnels,product_view_flag,product_view_first_timestamp,add_to_cart_flag,add_to_cart_first_timestamp,add_payment_info_flag,add_payment_info_first_timestamp,add_shipping_info_flag,add_shipping_info_first_timestamp,begin_checkout_flag,begin_checkout_first_timestamp,purchase_flag,purchase_first_timestamp
109957,2025-12,2025-12-19,1109514082.18,1766159058,"{'product_view': {'flag': 1, 'first_timestamp': '2025-12-19 21:14:22 UTC+0530'}, 'add_to_cart': ...",1,2025-12-19 21:14:22 UTC+0530,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0


In [13]:
funnel_flat.drop(columns=['funnels'],inplace=True)

In [14]:
funnel_flat.shape

(360104, 16)

In [15]:
funnel_flat=funnel_flat.merge(rest[['device','landing_page_type','source','medium','campaign','date_ist','user_id','session_id']],how='left',on=['date_ist','user_id','session_id'])

In [16]:
funnel_flat.sample()

,month,date_ist,user_id,session_id,product_view_flag,product_view_first_timestamp,add_to_cart_flag,add_to_cart_first_timestamp,add_payment_info_flag,add_payment_info_first_timestamp,add_shipping_info_flag,add_shipping_info_first_timestamp,begin_checkout_flag,begin_checkout_first_timestamp,purchase_flag,purchase_first_timestamp,device,landing_page_type,source,medium,campaign
153769,2025-12,2025-12-26,1031055432.18,1766749487,1,2025-12-26 17:14:49 UTC+0530,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,mobile,account,google,cpc,Google_Shopping_Selected_Products_MC_12082025


In [17]:
funnel_flat.shape

(360186, 21)

##### bug_ses

In [18]:
#sessions without checkouts but purchases exist
bug_ses=funnel_flat[(funnel_flat['purchase_flag']==1)&((funnel_flat['begin_checkout_flag']==0)| (funnel_flat['add_payment_info_flag']==0)|(funnel_flat['add_shipping_info_flag']==0))]

In [19]:
bug_ses=bug_ses.groupby(['date_ist','device','landing_page_type','source','medium','campaign']).agg(
    users=('user_id','nunique'),
    sessions=('session_id','nunique'),
    product_view=('session_id',lambda x: x[funnel_flat.loc[x.index, 'product_view_flag'] == 1].nunique()),
    add_to_cart=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_to_cart_flag'] == 1].nunique()),
    begin_checkout=('session_id',lambda x: x[funnel_flat.loc[x.index, 'begin_checkout_flag'] == 1].nunique()),
    add_payment_info=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_payment_info_flag'] == 1].nunique()),
    add_shipping_info=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_shipping_info_flag'] == 1].nunique()),
    purchase=('session_id',lambda x: x[funnel_flat.loc[x.index, 'purchase_flag'] == 1].nunique())
).reset_index()
# bug_ses

##### Is null values

In [20]:
funnel_flat.isnull().sum(), funnel_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360186 entries, 0 to 360185
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype    
---  ------                             --------------   -----    
 0   month                              360186 non-null  period[M]
 1   date_ist                           360186 non-null  object   
 2   user_id                            360186 non-null  float64  
 3   session_id                         360186 non-null  int64    
 4   product_view_flag                  360186 non-null  int64    
 5   product_view_first_timestamp       360186 non-null  object   
 6   add_to_cart_flag                   360185 non-null  float64  
 7   add_to_cart_first_timestamp        360185 non-null  object   
 8   add_payment_info_flag              360185 non-null  float64  
 9   add_payment_info_first_timestamp   360185 non-null  object   
 10  add_shipping_info_flag             360185 non-null  float64  
 11  add_shipping_

(month                    0
 date_ist                 0
 user_id                  0
 session_id               0
 product_view_flag        0
                      ...  
 device               22679
 landing_page_type    22679
 source               40744
 medium               41025
 campaign             41035
 Length: 21, dtype: int64,
 None)

In [21]:
grp_cols = ['device','landing_page_type','source','medium','campaign']

funnel_flat[grp_cols] = funnel_flat[grp_cols].fillna('Unknown')

In [22]:
funnel_data=funnel_flat.groupby(['date_ist','device','landing_page_type','source','medium','campaign']).agg(
    users=('user_id','nunique'),
    sessions=('session_id','nunique'),
    product_view=('session_id',lambda x: x[funnel_flat.loc[x.index, 'product_view_flag'] == 1].nunique()),
    add_to_cart=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_to_cart_flag'] == 1].nunique()),
    begin_checkout=('session_id',lambda x: x[funnel_flat.loc[x.index, 'begin_checkout_flag'] == 1].nunique()),
    add_payment_info=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_payment_info_flag'] == 1].nunique()),
    add_shipping_info=('session_id',lambda x: x[funnel_flat.loc[x.index, 'add_shipping_info_flag'] == 1].nunique()),
    purchase=('session_id',lambda x: x[funnel_flat.loc[x.index, 'purchase_flag'] == 1].nunique())
).reset_index()

In [23]:
# funnel_data

##### Purchases

In [24]:
rest.sample()

,date_ist,user_id,session_id,month,landing_page,landing_page_type,source,medium,campaign,device,orders,orders_dups_inclusive,rev,aov
212963,2025-12-31,226708770.18,1767121350,2025-12,https://keralaayurveda.com/products/dhanwantharam-gulika,products,None,None,None,mobile,0.00,0.00,0.00,0.00


In [25]:
rest.rev.sum()

np.float64(5483932.51)

In [26]:
# Problem: Any row with even 1 null among these columns is excluded entirely

In [27]:
grp_cols = ['device','landing_page_type','source','medium','campaign']

rest[grp_cols] = rest[grp_cols].fillna('Unknown')

In [28]:
payments=rest.groupby(['date_ist','device','landing_page_type','source','medium','campaign']).agg(
    users=('user_id','nunique'),
    sessions=('session_id','nunique'),
    payers=('user_id',lambda x: x[rest.loc[x.index, 'orders']!=0].nunique()),
    orders=('orders','sum'),
    orders_dups_inclusive=('orders_dups_inclusive','sum'),
    rev=('rev','sum')
).reset_index()

In [29]:
payments['aov']=payments['rev']/payments['orders']

In [30]:
payments.fillna(0,inplace=True)

In [31]:
payments.sample()

,date_ist,device,landing_page_type,source,medium,campaign,users,sessions,payers,orders,orders_dups_inclusive,rev,aov
6545,2025-12-20,mobile,products,wishlink,wishlink.com,5AiXDvbCvQMYeiEyBohqVA_p29415796,1,1,0,0.00,0.00,0.00,0.00


In [39]:
payments.orders_dups_inclusive.sum()

np.float64(8871.0)

In [37]:
funnel_data.sessions.sum()

np.int64(357695)

In [40]:
prefix="gold-tier/"
s3_path=f"s3://{bucket}/{prefix}payments.csv"
payments.to_csv(
    s3_path,
    index=False
)
s3_path=f"s3://{bucket}/{prefix}funnel.csv"
funnel_data.to_csv(
    s3_path,
    index=False
)
s3_path=f"s3://{bucket}/{prefix}sess_bug.csv"
bug_ses.to_csv(
    s3_path,
    index=False
)

In [41]:
check=payments.groupby('date_ist').agg(
    orders=('orders','sum'),
    ext=('orders_dups_inclusive','sum')
).reset_index()

In [42]:
check[check['ext']>check['orders']]

,date_ist,orders,ext
25,2025-12-26,277.00,278.00
